<a href="https://colab.research.google.com/github/busraminal/Multimodal-Deepfake-Tespit-Sistemi/blob/main/df_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Found existing installation: torch 2.9.1+cpu
Uninstalling torch-2.9.1+cpu:
  Successfully uninstalled torch-2.9.1+cpu
Found existing installation: torchvision 0.24.1+cpu
Uninstalling torchvision-0.24.1+cpu:
  Successfully uninstalled torchvision-0.24.1+cpu
Found existing installation: torchaudio 2.9.1+cpu
Uninstalling torchaudio-2.9.1+cpu:
  Successfully uninstalled torchaudio-2.9.1+cpu
Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.9.1%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.24.1%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (5.9 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.9.1%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (6.9 kB)
Using cached https://download.pytorch.org/whl/cpu/torch-2.9.1%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl (184.4 MB)
Using cached https://download.pytorch.org/whl/c

In [2]:
!pip install fastapi uvicorn pydantic sentence-transformers faiss-cpu


In [3]:
import torch
print("torch:", torch.__version__)
print("cuda:", torch.cuda.is_available())


torch: 2.9.1+cpu
cuda: False


In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
print("sentence-transformers OK")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sentence-transformers OK


In [5]:
import faiss
import numpy as np

# basit test
d = 5
index = faiss.IndexFlatIP(d)

x = np.random.rand(3, d).astype("float32")
faiss.normalize_L2(x)
index.add(x)

D, I = index.search(x[:1], 2)
print("faiss OK", D, I)


faiss OK [[0.9999999 0.9402309]] [[0 2]]


In [6]:
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# 1) RAG verisini yükle
with open("rag_knowledge.json", encoding="utf-8") as f:
    rag_data = json.load(f)

texts = [f"[{d['category']}] {d['text']}" for d in rag_data]

print("RAG entries:", len(texts))

# 2) Embedding modeli
emb_model = SentenceTransformer("all-MiniLM-L6-v2")

embs = emb_model.encode(texts, normalize_embeddings=True)
embs = np.array(embs, dtype="float32")

# 3) FAISS index
index = faiss.IndexFlatIP(embs.shape[1])
index.add(embs)

print("Index size:", index.ntotal)


RAG entries: 9
Index size: 9


In [7]:
def retrieve(query, k=3):
    q = emb_model.encode([query], normalize_embeddings=True)
    q = np.array(q, dtype="float32")
    _, I = index.search(q, k)
    return [texts[i] for i in I[0]]

# test sorgusu
results = retrieve("lip sync uyumsuzlugu deepfake", k=3)

print("Retrieved:")
for r in results:
    print("-", r)


Retrieved:
- [context] Sessiz videolarda deepfake tespiti ağırlıklı olarak görsel ve biyomekanik ipuçlarına dayanır. Ses ve lip-sync skorları bu senaryoda ikincil öneme sahiptir.
- [Sl] Sessiz veya az konuşmalı videolarda lip-sync analizi güvenilir değildir. Bu durumda dudak-ses uyumsuzluğu skorları değerlendirme dışı bırakılmalıdır.
- [Sl] Ses ve dudak hareketleri arasındaki zamansal uyumsuzluk, deepfake videolarda sık görülen bir bulgudur. Özellikle 60–100 ms üzerindeki gecikmeler, yapay konuşma senkronizasyonuna işaret edebilir.


In [8]:
from fastapi import FastAPI, Header, HTTPException
from pydantic import BaseModel
from typing import List, Optional
import threading, uvicorn

app = FastAPI()

APP_TOKEN = "mina-secret"

class AnalyzeReq(BaseModel):
    video_id: str
    scores: List[float]  # [Sv, Sl, Sb, Sh, Sa, Sf]

class AnalyzeResp(BaseModel):
    confidence: float
    explanation: str

@app.post("/analyze", response_model=AnalyzeResp)
def analyze(
    req: AnalyzeReq,
    x_token: Optional[str] = Header(default=None)
):
    if x_token != APP_TOKEN:
        raise HTTPException(status_code=401, detail="Unauthorized")

    Sv, Sl, Sb, Sh, Sa, Sf = req.scores

    # 🔹 RAG query (feature-aware, basit versiyon)
    if Sa < 0.1 and Sl < 0.1:
        query = "silent video deepfake detection rules"
    elif Sl > 0.5:
        query = "lip sync mismatch deepfake"
    else:
        query = "multimodal deepfake forensic explanation"

    ctx_list = retrieve(query, k=3)
    ctx = "\n".join(ctx_list)

    explanation = f"""
SYSTEM:
You are an AI assistant explaining a multimodal deepfake analysis to a non-technical user.
Answer the user's question clearly and concisely.
Do NOT repeat the analysis summary verbatim.
Use the scores only as justification.

USER QUESTION:
{question}

ANALYSIS SCORES:
Sv={Sv}, Sl={Sl}, Sb={Sb}, Sh={Sh}, Sa={Sa}, Sf={Sf}

CONTEXT:
{rag_context}

""".strip()

    return AnalyzeResp(confidence=Sf, explanation=explanation)

# ---- RUN SERVER ----
threading.Thread(
    target=lambda: uvicorn.run(app, host="0.0.0.0", port=8000),
    daemon=True
).start()

print("FastAPI + RAG server running on port 8000")


FastAPI + RAG server running on port 8000


In [9]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64


INFO:     Started server process [4228]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [10]:
!cloudflared tunnel --url http://localhost:8000



/bin/bash: line 1: cloudflared: command not found


In [11]:
!nohup ./cloudflared-linux-amd64 tunnel --url http://127.0.0.1:8000 --no-autoupdate > tunnel.log 2>&1 &


In [12]:
!grep trycloudflare tunnel.log


In [13]:
!pkill -f uvicorn
!pkill -f cloudflared || true


^C


In [25]:
%%writefile server.py
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Optional

app = FastAPI()

class AnalyzeReq(BaseModel):
    video_id: str
    scores: List[float]
    question: Optional[str] = None

@app.get("/health")
def health():
    return {"ok": True}

@app.post("/analyze")
def analyze(req: AnalyzeReq):
    question = req.question or "Bu video neden gerçek sayıldı?"

    return {
        "video_id": req.video_id,
        "answer": (
            f"Soru: {question}\n"
            f"Skorlar: {req.scores}\n"
            "Sonuç: Skorlar gerçek videolarla daha uyumludur."
        )
    }


Writing server.py


In [26]:
!ls


cloudflared-linux-amd64    rag_knowledge.json  server.py
cloudflared-linux-amd64.1  sample_data	       tunnel.log


In [5]:
!nohup uvicorn server:app --host 0.0.0.0 --port 8000 > server.log 2>&1 &





In [6]:
!curl http://127.0.0.1:8000/health


{"ok":true}

In [7]:
!nohup ./cloudflared-linux-amd64 tunnel \
  --url http://127.0.0.1:8000 \
  --no-autoupdate > tunnel.log 2>&1 &


In [8]:
!grep trycloudflare tunnel.log | tail -n 1


2025-12-28T20:49:10Z INF |  https://talented-federal-twenty-scsi.trycloudflare.com                                    |


In [9]:
import requests

url = "https://talented-federal-twenty-scsi.trycloudflare.com/analyze"

payload = {
    "video_id": "pc_test_001",
    "scores": [0.82, 0.15, 0.60, 0.55, 0.02, 0.78],
    "question": "Bu video neden gerçek sayıldı?"
}

r = requests.post(url, json=payload, timeout=15)
print("STATUS:", r.status_code)
print("TEXT:", r.text)


STATUS: 200
TEXT: {"video_id":"pc_test_001","answer":"Soru: Bu video neden gerçek sayıldı?\nSkorlar: [0.82, 0.15, 0.6, 0.55, 0.02, 0.78]\nSonuç: Skorlar gerçek videolarla daha uyumludur."}
